In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df = spark.read.load("./marketing_campaign.csv",
                format="csv",
                sep="\t",
                inferchema="true",
                header="true")

In [3]:
df.show()

24/08/07 10:32:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

In [4]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Year_Birth: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Kidhome: string (nullable = true)
 |-- Teenhome: string (nullable = true)
 |-- Dt_Customer: string (nullable = true)
 |-- Recency: string (nullable = true)
 |-- MntWines: string (nullable = true)
 |-- MntFruits: string (nullable = true)
 |-- MntMeatProducts: string (nullable = true)
 |-- MntFishProducts: string (nullable = true)
 |-- MntSweetProducts: string (nullable = true)
 |-- MntGoldProds: string (nullable = true)
 |-- NumDealsPurchases: string (nullable = true)
 |-- NumWebPurchases: string (nullable = true)
 |-- NumCatalogPurchases: string (nullable = true)
 |-- NumStorePurchases: string (nullable = true)
 |-- NumWebVisitsMonth: string (nullable = true)
 |-- AcceptedCmp3: string (nullable = true)
 |-- AcceptedCmp4: string (nullable = true)
 |-- AcceptedCmp5: string (nullab

In [5]:
df.count()

2240

In [6]:
df.first()

Row(ID='5524', Year_Birth='1957', Education='Graduation', Marital_Status='Single', Income='58138', Kidhome='0', Teenhome='0', Dt_Customer='04-09-2012', Recency='58', MntWines='635', MntFruits='88', MntMeatProducts='546', MntFishProducts='172', MntSweetProducts='88', MntGoldProds='88', NumDealsPurchases='3', NumWebPurchases='8', NumCatalogPurchases='10', NumStorePurchases='4', NumWebVisitsMonth='7', AcceptedCmp3='0', AcceptedCmp4='0', AcceptedCmp5='0', AcceptedCmp1='0', AcceptedCmp2='0', Complain='0', Z_CostContact='3', Z_Revenue='11', Response='1')

In [7]:
dfSelected = df.select(
    col("ID").alias("id"),
    col("Year_Birth").alias("year_birth"),
    col("Education").alias("education"),
    col("Kidhome").alias("count_kid"),
    col("Teenhome").alias("count_teen"),
    col("Dt_Customer").alias("date_customer"),
    col("Recency").alias("days_last_login")
)
dfSelected.show()

+----+----------+----------+---------+----------+-------------+---------------+
|  id|year_birth| education|count_kid|count_teen|date_customer|days_last_login|
+----+----------+----------+---------+----------+-------------+---------------+
|5524|      1957|Graduation|        0|         0|   04-09-2012|             58|
|2174|      1954|Graduation|        1|         1|   08-03-2014|             38|
|4141|      1965|Graduation|        0|         0|   21-08-2013|             26|
|6182|      1984|Graduation|        1|         0|   10-02-2014|             26|
|5324|      1981|       PhD|        1|         0|   19-01-2014|             94|
|7446|      1967|    Master|        0|         1|   09-09-2013|             16|
| 965|      1971|Graduation|        0|         1|   13-11-2012|             34|
|6177|      1985|       PhD|        1|         0|   08-05-2013|             32|
|4855|      1974|       PhD|        1|         0|   06-06-2013|             19|
|5899|      1950|       PhD|        1|  

In [8]:
dfSelected.printSchema()

root
 |-- id: string (nullable = true)
 |-- year_birth: string (nullable = true)
 |-- education: string (nullable = true)
 |-- count_kid: string (nullable = true)
 |-- count_teen: string (nullable = true)
 |-- date_customer: string (nullable = true)
 |-- days_last_login: string (nullable = true)



In [9]:
df.rdd.id()

31

In [12]:
dfSelected.describe().show()

+-------+------------------+------------------+---------+-------------------+------------------+-------------+-----------------+
|summary|                id|        year_birth|education|          count_kid|        count_teen|date_customer|  days_last_login|
+-------+------------------+------------------+---------+-------------------+------------------+-------------+-----------------+
|  count|              2240|              2240|     2240|               2240|              2240|         2240|             2240|
|   mean| 5592.159821428571|1968.8058035714287|     null|0.44419642857142855|           0.50625|         null|        49.109375|
| stddev|3246.6621975643416|11.984069456885827|     null| 0.5383980977345935|0.5445382307698761|         null|28.96245280837821|
|    min|                 0|              1893| 2n Cycle|                  0|                 0|   01-01-2013|                0|
|    max|              9999|              1996|      PhD|                  2|                 2| 

In [13]:
dfConverted1 = dfSelected.withColumn("count_children", coalesce("count_kid", lit(0)) + coalesce("count_teen", lit(0)))
dfConverted1.show()

+----+----------+----------+---------+----------+-------------+---------------+--------------+
|  id|year_birth| education|count_kid|count_teen|date_customer|days_last_login|count_children|
+----+----------+----------+---------+----------+-------------+---------------+--------------+
|5524|      1957|Graduation|        0|         0|   04-09-2012|             58|           0.0|
|2174|      1954|Graduation|        1|         1|   08-03-2014|             38|           2.0|
|4141|      1965|Graduation|        0|         0|   21-08-2013|             26|           0.0|
|6182|      1984|Graduation|        1|         0|   10-02-2014|             26|           1.0|
|5324|      1981|       PhD|        1|         0|   19-01-2014|             94|           1.0|
|7446|      1967|    Master|        0|         1|   09-09-2013|             16|           1.0|
| 965|      1971|Graduation|        0|         1|   13-11-2012|             34|           1.0|
|6177|      1985|       PhD|        1|         0| 

In [14]:
dfConverted1.select(
    col("id"),
    col("count_kid"),
    col("count_teen"),
    col("count_children")
).limit(5).show()

+----+---------+----------+--------------+
|  id|count_kid|count_teen|count_children|
+----+---------+----------+--------------+
|5524|        0|         0|           0.0|
|2174|        1|         1|           2.0|
|4141|        0|         0|           0.0|
|6182|        1|         0|           1.0|
|5324|        1|         0|           1.0|
+----+---------+----------+--------------+



In [16]:
dfConverted1\
    .select("education")\
    .distinct()\
    .show()

+----------+
| education|
+----------+
|  2n Cycle|
|       PhD|
|    Master|
|Graduation|
|     Basic|
+----------+



In [18]:
educationInvalid = '2n Cycle'
educationDefault = 'NONE'
dfConverted2 = dfConverted1.withColumn(
    "education",
    when(col("education") == lit(educationInvalid), educationDefault).otherwise(col("education"))
)
dfConverted2.select("education").distinct().show()

+----------+
| education|
+----------+
|       PhD|
|    Master|
|Graduation|
|     Basic|
|      NONE|
+----------+



In [24]:
dfConverted2.select(col("education")).distinct().show()

+----------+
| education|
+----------+
|       PhD|
|    Master|
|Graduation|
|     Basic|
|      NONE|
+----------+



In [31]:
educationInvalid = '2n Cycle'
educationDefault = 'NONE'
dfConverted1.selectExpr(
    "*",
    f"CASE WHEN education == '{educationInvalid}' THEN '{educationDefault}' ELSE education END as education"
).show()

+----+----------+----------+---------+----------+-------------+---------------+--------------+----------+
|  id|year_birth| education|count_kid|count_teen|date_customer|days_last_login|count_children| education|
+----+----------+----------+---------+----------+-------------+---------------+--------------+----------+
|5524|      1957|Graduation|        0|         0|   04-09-2012|             58|           0.0|Graduation|
|2174|      1954|Graduation|        1|         1|   08-03-2014|             38|           2.0|Graduation|
|4141|      1965|Graduation|        0|         0|   21-08-2013|             26|           0.0|Graduation|
|6182|      1984|Graduation|        1|         0|   10-02-2014|             26|           1.0|Graduation|
|5324|      1981|       PhD|        1|         0|   19-01-2014|             94|           1.0|       PhD|
|7446|      1967|    Master|        0|         1|   09-09-2013|             16|           1.0|    Master|
| 965|      1971|Graduation|        0|        

In [36]:
# date_customer컬럼에서 6년을 더한 컬럼 생성
dfWithJoined = dfConverted2.withColumn("date_joined", add_months(to_date(col("date_customer"), "d-M-yyyy"),72))
dfWithJoined.limit(3).show()

+----+----------+----------+---------+----------+-------------+---------------+--------------+-----------+
|  id|year_birth| education|count_kid|count_teen|date_customer|days_last_login|count_children|date_joined|
+----+----------+----------+---------+----------+-------------+---------------+--------------+-----------+
|5524|      1957|Graduation|        0|         0|   04-09-2012|             58|           0.0| 2018-09-04|
|2174|      1954|Graduation|        1|         1|   08-03-2014|             38|           2.0| 2020-03-08|
|4141|      1965|Graduation|        0|         0|   21-08-2013|             26|           0.0| 2019-08-21|
+----+----------+----------+---------+----------+-------------+---------------+--------------+-----------+

